### Installing Dependencies

In [ ]:
%pip install langchain
%pip install langchain-ollama
%pip install python-dotenv

In [ ]:
#pip install -U langchain langchain-openai

### Load ENV

In [1]:
from dotenv import load_dotenv
import os
load = load_dotenv('./../.env')

print(os.getenv("LANGSMITH_API_KEY"))

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


### Interacting with LLM

In [2]:
from langchain_ollama import ChatOllama

llm =  ChatOllama(
    base_url = "http://localhost:11434",
    model = "deepseek-r1:8b",
    temperature= 0.5 ,
    max_tokens = 1000
)

response = llm.invoke("hello, How are you doing?")

print(response)

content='<think>\nAlright, so I\'m trying to figure out how to respond to the user\'s greeting. They said, "Hello, How are you doing?" and then I responded with a greeting in return. Now, I need to think about how to continue this conversation.\n\nFirst, I should probably mirror their greeting by asking them how they\'re doing. That makes sense because it shows politeness and engagement. So, my next step is to say something like, "I\'m doing well, thank you! How can I assist you today?"\n\nBut wait, maybe I should consider the tone. Since they started with a friendly greeting, keeping the response warm and open would be good. I don\'t want it to sound too formal or robotic.\n\nAlso, after that, I should offer help. The user might have a specific question or need assistance with something. So, adding "How can I assist you today?" at the end of my response makes sense because it invites them to share what they need help with.\n\nI should make sure there are no grammatical errors in my re

#### Structured output

In [3]:
import re
import json
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage # Import AIMessage for type hinting (optional but good practice)

# --- Initialize the LLM (same as before) ---
llm =  ChatOllama(
    base_url = "http://localhost:11434",
    model = "deepseek-r1:8b",
    temperature= 0.5
)

# --- Invoke the LLM (same as before) ---
prompt = "What are AI agents? What is the difference between AI, AI agents, AI workflow, automation etc"
print(f"Sending prompt: \"{prompt}\"\n")
response: AIMessage = llm.invoke(prompt) # Added type hint


# --- Structure and Visualize the Output ---

# 1. Extract the full content string
full_content = response.content

# 2. Separate the <think> block and the main response using regex
think_content = "N/A" # Default if not found
main_response = full_content # Default if no think block

# Use regex to find the think block (dot matches newline)
# Assuming the model *might* output <think> tags based on the original code's intent
# If your model doesn't use <think>, this part might not find anything, which is fine.
match = re.search(r"<think>(.*?)</think>", full_content, re.DOTALL)
if match:
    think_content = match.group(1).strip() # Get content inside tags
    # Find the end of the </think> tag to get the main response after it
    end_think_tag_index = match.end()
    main_response = full_content[end_think_tag_index:].strip()
else:
    # If no <think> block, assume the whole content is the main response
    # and the thinking process is not explicitly separated in the output.
    main_response = full_content.strip()
    think_content = "(No <think> block found in response)"


# 3. Extract Metadata
response_meta = response.response_metadata
usage_meta = response.usage_metadata
other_info = {
    "id": response.id,
    "additional_kwargs": response.additional_kwargs,
    # Add type if needed, though it's usually clear from context
    "type": response.type
}

# --- FIX: Define a default serializer function for json.dumps ---
def default_serializer(obj):
    """Convert non-serializable objects to strings."""
    try:
        # Let the default encoder try first
        return json.JSONEncoder().default(obj)
    except TypeError:
        # If it fails, convert the object to its string representation
        return str(obj)

# 4. Print the structured output
print("="*50)
print("--- Thinking Process ---")
print("="*50)
print(think_content)
print("\n") # Add a newline for spacing

print("="*50)
print("--- Main Response ---")
print("="*50)
print(main_response)
print("\n")

print("="*50)
print("--- Response Metadata ---")
print("="*50)

# --- FIX: Use the default serializer in json.dumps ---
print(json.dumps(response_meta, indent=4, default=default_serializer))
print("\n")

print("="*50)
print("--- Usage Metadata ---")
print("="*50)
# Assuming usage_meta is already JSON serializable (usually is)
# If you get an error here too, apply the same default=default_serializer fix
print(json.dumps(usage_meta, indent=4))
print("\n")

print("="*50)
print("--- Other Info ---")
print("="*50)
# other_info should be serializable as constructed
print(json.dumps(other_info, indent=4))
print("="*50)


Sending prompt: "What are AI agents? What is the difference between AI, AI agents, AI workflow, automation etc"

--- Thinking Process ---
Okay, so I'm trying to understand what AI agents are. From what I remember, AI stands for Artificial Intelligence, which is about machines being able to perform tasks that usually require human intelligence. But then there's this term "AI agent," and I'm not entirely sure how it fits into all of this.

I think an AI agent might be some kind of software program, right? Maybe like a robot or something that can make decisions on its own. But wait, isn't AI more about learning and adapting, while agents are more like autonomous entities? Hmm, maybe I should break it down further.

So, if AI is the broader field, then AI agents must be a specific type of system within AI. Maybe they're like the ones that take inputs and process them to make decisions or actions without human intervention. For example, chatbots are AI agents because they interact with user